In [1]:
import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

plc_buffer_name = 'Main.profiler_buffer'
plc_numlayers_name = 'Main.FB_NN.nn.num_layers'
plc_numneurons_name = 'Main.FB_NN.nn.layers[[index]].num_neurons'
plc_MeasureFinished_name = 'Main.MeasureFinished'

num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]

In [11]:



measurements = []
with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
    for i in num_layers:
        plc.write_by_name(plc_numlayers_name,i,pyads.PLCTYPE_UINT)
        for j in num_neurons:
            for k in range(i-1):
                plc.write_by_name(plc_numneurons_name.replace('[index]',f'{k+1}'),j,pyads.PLCTYPE_UINT)
            plc.write_by_name(plc_MeasureFinished_name, False ,pyads.PLCTYPE_BOOL)
            while(1):
                plc_MeasureFinished = plc.read_by_name(plc_MeasureFinished_name, pyads.PLCTYPE_BOOL)
                if plc_MeasureFinished:
                    profiler_buffer = plc.read_by_name(plc_buffer_name, pyads.PLCTYPE_DWORD*1000)
                    print(f'num_layers:{i}, num_neurouns:{j}\n')
                    print(profiler_buffer[0:10])
                    for k in profiler_buffer:
                        measurements.append([i,j,k])
                    break

df = pd.DataFrame(measurements, columns=['num_layers', 'num_neurons', 'measurement'])

num_layers:6, num_neurouns:500

[3787, 3723, 3728, 3701, 3678, 3663, 3731, 3758, 3730, 3749]
num_layers:6, num_neurouns:400

[2399, 2353, 2400, 2391, 2438, 2401, 2345, 2359, 2331, 2369]
num_layers:6, num_neurouns:300

[1345, 1369, 1350, 1347, 1363, 1338, 1357, 1365, 1374, 1375]
num_layers:6, num_neurouns:200

[600, 607, 607, 626, 611, 606, 613, 599, 598, 608]
num_layers:6, num_neurouns:100

[160, 160, 160, 160, 160, 160, 160, 160, 160, 160]
num_layers:6, num_neurouns:10

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
num_layers:5, num_neurouns:500

[2780, 2758, 2830, 2764, 2727, 2763, 2735, 2708, 2715, 2744]
num_layers:5, num_neurouns:400

[1766, 1783, 1773, 1782, 1748, 1758, 1753, 1738, 1745, 1758]
num_layers:5, num_neurouns:300

[990, 999, 1007, 995, 1032, 1005, 1012, 1001, 998, 999]
num_layers:5, num_neurouns:200

[455, 452, 454, 452, 452, 453, 452, 459, 453, 453]
num_layers:5, num_neurouns:100

[121, 121, 121, 121, 121, 121, 121, 121, 121, 121]
num_layers:5, num_neurouns:10

[3, 2, 3, 3, 2, 2, 3, 

In [12]:
df.to_pickle('exe_time_measurement_sigmoid_arch.pkl')

In [13]:
color_map = {6: 'rgb(255,0,0)', 5: 'rgb(0,255,0)', 4: 'rgb(0,0,255)', 3: 'rgb(255,255,0)'}

fig = px.box(df, x="num_neurons", y="measurement", color="num_layers",boxmode = 'overlay', color_discrete_map=color_map)
fig.update_layout(xaxis_title="Number of Neurons in each layer", yaxis_title="Execution Time [us]", legend_title = 'Number of Layers')
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = num_neurons
    )
)

for i in num_layers:
    mean = []
    for j in num_neurons:
        mean.append(df.loc[(df['num_layers'] == i) & (df['num_neurons'] == j)]['measurement'].mean())
    fig.add_trace(go.Scatter(x=num_neurons, y=mean, showlegend=False,  marker_color=color_map[i] ))
fig.show()